## SQL Databases

In [1]:
## Create sample SQLite Database
import sqlite3
import os

os.makedirs("data/databases",exist_ok=True)

In [2]:
## Create Sample Database
conn = sqlite3.connect("data/databases/company.db")
cursor = conn.cursor()

In [3]:
## Create tables
cursor.execute(''' CREATE TABLE IF NOT EXISTS employees
            (id INTEGER PRIMARY KEY,name TEXT,roleTEXT,department TEXT,salary REAL)''')

In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
                 (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [5]:
# Insert sample data
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]


In [6]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [7]:
cursor.execute("Select * from employees")

In [8]:
conn.commit()
conn.close()


## Database Content Extraction

In [9]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [10]:
## MEthod1 SQLDATABASE Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

## get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL:")
print(db.get_table_info())

Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	department	salary
1	John Doe	Engineering	95000.0
2	Jane Smith	Analytics	105000.0
3	Mike Johnson	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


In [13]:
import sqlite3
from typing import List
from langchain_core.documents import Document

print("\n2️⃣ Custom SQL Processing")

def get_columns(cursor, table_name):
    cursor.execute(f"PRAGMA table_info({table_name});")
    return [col[1] for col in cursor.fetchall()]


def sql_to_documents(db_path: str) -> List[Document]:
    """Convert SQL Database to Documents with context (RAG-safe)"""
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents = []

    # -------- Strategy 1: Table-level documents --------
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [t[0] for t in cursor.fetchall()]

    for table_name in tables:
        columns = get_columns(cursor, table_name)
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        table_content = f"""Table: {table_name}
Columns: {', '.join(columns)}
Total Records: {len(rows)}

Sample Records:
"""

        for row in rows[:5]:
            record = dict(zip(columns, row))
            table_content += f"{record}\n"

        documents.append(
            Document(
                page_content=table_content,
                metadata={
                    "source": db_path,
                    "table_name": table_name,
                    "num_records": len(rows),
                    "data_type": "sql_table"
                }
            )
        )

    # -------- Strategy 2: Relationship document (SAFE) --------
    if "employees" in tables and "projects" in tables:
        emp_cols = get_columns(cursor, "employees")

        role_col = (
            "role" if "role" in emp_cols else
            "designation" if "designation" in emp_cols else
            "position" if "position" in emp_cols else None
        )

        role_expr = f"e.{role_col}" if role_col else "'N/A'"

        query = f"""
        SELECT e.name, {role_expr} AS role, 
               p.name AS project_name, p.status
        FROM employees e
        JOIN projects p ON e.id = p.lead_id
        """

        cursor.execute(query)
        relationships = cursor.fetchall()

        if relationships:
            rel_content = "Employee–Project Relationships:\n\n"
            for name, role, project, status in relationships:
                rel_content += f"{name} ({role}) leads {project} – Status: {status}\n"

            documents.append(
                Document(
                    page_content=rel_content,
                    metadata={
                        "source": db_path,
                        "data_type": "sql_relationships",
                        "query": "employee_project_join"
                    }
                )
            )

    conn.close()
    return documents



2️⃣ Custom SQL Processing


In [14]:
sql_to_documents("data/databases/company.db")

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, roleTEXT, department, salary\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'John Doe', 'roleTEXT': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'roleTEXT': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'roleTEXT': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Sarah Williams', 'roleTEXT': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active